![Königsweg Logo](../img/koenigsweg_150.png)

<span style="font-size: small;float: right;">&copy; 2015-2020 Alexander C.S. Hendorf, <a href="http://koenigsweg.com">Königsweg GmbH</a>, Mannheim </span>

---

# Analytics with  Pandas and Jupyterlab

---

# Data Aggregation with Pandas

---

In [ ]:
import numpy as np
import pandas as pd
import random
from IPython.core.display import Image, display

### Aggregation

Pandas logic was inspired by SQL.

In [ ]:
sales_data = pd.read_excel('../data/blooth_sales_data_clean.xlsx')
sales_data['turnover'] = sales_data['units'] * sales_data['unitprice']
sales_data.head(5)

In [ ]:
sales_data.groupby('customer').count()[:5]

In [ ]:
sales_data.groupby('customer').count().sort_values('units', ascending=False)[:5]

In [ ]:
sales_data.groupby(['customer', 'product']).sum().sort_values('turnover', ascending=False)[:5]

In [ ]:
sales_data.groupby(['customer', 'product']).sum().index

See more on Multi-Indexes below

---

#### Execise

Find the product with the lowest turnover of all.

In [ ]:
# Your code here


In [ ]:
# %load ../solutions/501.py

Indentify the best sold product per customer with the lowest turnover.

In [ ]:
# Your code here


In [ ]:
# %load ../solutions/502.py


---

**`.agg()`** aka. **`.aggreagte()`**

In [ ]:
sales_data.head(5)

In [ ]:
sales_data['turnover'].agg([sum, min, max])

In [ ]:
sales_data[['turnover', 'units']].agg({'turnover': 'min', 'units': 'max'})

In [ ]:
def myfunc(series):
    return series.sum()/series.count()

sales_data[['turnover', 'units']].agg(myfunc)

---

#### Excercise

Calculate the median of *turnover* using agg

\begin{equation*} \tilde{x} = \begin{cases} x_{\frac{n+1}{2}} & \text{for } n \text{ odd}\\ \frac{1}{2}\left(x_{\frac{n}{2}} + x_{\frac{n}{2}+1}\right) & \text{for } n \text{ even} \end{cases} \end{equation*}

In [ ]:
# Your code here


In [ ]:
# %load ../solutions/503.py

Calculate the median birthday using agg.<br>
Hint: you can add datetime.timedelta(days=n) to datetime.datetime()

In [ ]:
# Your code here


In [ ]:
# %load ../solutions/504.py

---

### Pivoting

Another possibiliy to aggreagate data, you might know the pivoting from Excel  

In [ ]:
sales_data.head(5)

In [ ]:
pvt = pd.pivot_table(sales_data,
               values='turnover',
               index=['customer', 'product'],
               aggfunc=np.sum,
                      )

In [ ]:
type(pvt), pvt.index

---

### Reshaping with **`.melt()`**

In [ ]:
data = pd.DataFrame({
    'first': ['John', 'Mary'],
    'last': ['Wayne', 'Martin'],
    'latin': [5.5, 6.0],
    'greek': [130, 150],
    'math': [130, 150],
})
data

In [ ]:
data.melt(id_vars=['first', 'last'])

---

**`.stack() / .unstack()`**

In [ ]:
sales_data.stack()

In [ ]:
sales_data.stack().unstack()

---

## Merging

In [ ]:
df = pd.read_json('../data/sampledf.json')
df.index = ["R{:02d}".format(i) for i in range(len(df))]
df.columns = ["C{:02d}".format(i) for i in range(len(df.columns))]
df

In [ ]:
add_df = pd.Series([random.randint(0, 100) for x in range(10)], 
                    index=["R{:02d}".format(i) for i in range(3, len(df)+3)])
add_df.name = 'C10'
add_df

In [ ]:
df['C10'] = add_df
df

In [ ]:
df.drop('C10', errors='ignore')

In [ ]:
df.drop('C10', errors='ignore', axis=1, inplace=True)
df

In [ ]:
pd.DataFrame(add_df)

#### Inner Joins

In [ ]:
display(Image("../img/join-inner.png", width=450, unconfined=True))
#<img src="../img/join-inner.png" alt="Drawing" style="float: left; width: 450px ;"/>

In [ ]:
df.join(pd.DataFrame(add_df), how='inner')
# inner: both indexes match

In [ ]:
df.drop('C10', errors='ignore', axis=1, inplace=True)  # cleanup

#### Outer Joins

In [ ]:
display(Image("../img/join-outer.png", width=450, unconfined=True))

In [ ]:
df.join(pd.DataFrame(add_df), how='outer')
# outer: combines everything - puts NaN if no match on either side

In [ ]:
df.drop('C10', errors='ignore', axis=1, inplace=True)  # cleanup

#### Left-Outer Joins

In [ ]:
display(Image("../img/join-left.png", width=450, unconfined=True))

In [ ]:
df.join(pd.DataFrame(add_df), how='left')
# left: match all that occur in left = df joined upon

In [ ]:
df.drop('C10', errors='ignore', axis=1, inplace=True)  # cleanup

#### Right-Outer Joins

In [ ]:
display(Image("../img/join-right.png", width=450, unconfined=True))

In [ ]:
df.join(pd.DataFrame(add_df), how='right')
# right: mtach all that occur in right=joined df

---

### Multi-Index

In [ ]:
# Create some demo data
somany = 20
dfm = pd.DataFrame({'city':[x for x in 
                            ['Paris', 'London', 'Berlin', 'New York', 'Mannheim']*10][:20],
      'category': [random.randint(1, 3) for x in range(somany)],
      'price': [random.randint(150, 300) for x in range(somany)],
      'rating': [random.randint(0, 10) for x in range(somany)]})
dfm['country'] = dfm['city'].map(
    {'Paris': 'FR', 'London': 'GB', 'Berlin': 'DE', 'New York': 'US', 'Mannheim': 'DE'})

In [ ]:
dfm

In [ ]:
dfg = dfm.groupby(['country', 'city', 'category']).mean()

In [ ]:
dfg

In [ ]:
dfg.index

In [ ]:
dfg.index.levels

In [ ]:
dfg.index.names

In [ ]:
dfg.index.values

In [ ]:
dfg.stack()

In [ ]:
pd.set_option('display.multi_sparse', False)
dfg

In [ ]:
pd.set_option('display.multi_sparse', True)

In [ ]:
dfg.index.get_level_values(2)

In [ ]:
dfg.index.get_level_values(1)

In [ ]:
dfg.loc['DE']

In [ ]:
dfg.loc[('FR', 'Paris')]

In [ ]:
dfg.loc[('FR', 'Paris')].max()

In [ ]:
dfg.loc[('DE', 'Mannheim', 1)]